In [2]:
from matplotlib import pyplot as plt
from tensorflow import keras
import cv2
import tensorflow as tf
import tensorflow.keras.backend as K
import os
import dlib
from imutils import face_utils
from sklearn.metrics import mean_squared_error as MSE
import numpy as np
import os
import cv2
import dlib


2021-12-18 18:56:56.653107: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-18 18:56:56.653130: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
## Load model
model = tf.keras.models.load_model("/home/data/data/kodeiri/ML_project/saved_model/my_model4")
## Load heatmap template for each class
HM_shape = (100,100,3) ## template heatmap shape DO NOT CHANGE, only if the original shape changed before saving

2021-12-18 18:56:59.322436: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-12-18 18:56:59.350165: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:c1:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.6705GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2021-12-18 18:56:59.350300: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-18 18:56:59.350354: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-12-18 18:56:59.350393: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic l

In [4]:
## Load detector libraries
detector = dlib.cnn_face_detection_model_v1('/home/data/data/kodeiri/ML_project/dogHeadDetector.dat')
predictor = dlib.shape_predictor('/home/data/data/kodeiri/ML_project/landmarkDetector.dat')

In [9]:
## heatmap loader
def Hm_loader(path):
    template_pre =np.loadtxt(path)
    template = template_pre.reshape(template_pre.shape[0], template_pre.shape[1] // 3, 3)
    return template


In [5]:
young_path = "/home/data/data/kodeiri/ML_project/HM/Young.txt"

In [6]:
adult_path = "/home/data/data/kodeiri/ML_project/HM/Adult.txt"

In [7]:
senior_path = "/home/data/data/kodeiri/ML_project/HM/Senior.txt"

In [10]:
Young_template = Hm_loader(young_path)

In [11]:
Adult_template = Hm_loader(adult_path)
smallest = Adult_template.min(axis=(0, 1))

largest = Adult_template.max(axis=(0, 1))
print(max(largest))
print(smallest, largest)

23.149321266968325
[0. 0. 0.] [ 1.71493213  4.29864253 23.14932127]


In [12]:
Senior_template = Hm_loader(senior_path)


In [ ]:
# wyszukiwanie zdjęcia na białym tle
def photo_detector(src_image):

    img_in = cv2.cvtColor(src_image, cv2.COLOR_BGR2RGB)
    # img = cv2.resize(img, dsize=None, fx=0.5, fy=0.5)
    
    gray = cv2.cvtColor(img_in, cv2.COLOR_BGR2GRAY)
    height, width = gray.shape

    ret, thresh = cv2.threshold(gray, 254, 255, 1)

    # f, axarr = plt.subplots(1,3)

    photo_width = np.count_nonzero(thresh == 255, axis=0).max()
    photo_height = np.count_nonzero(thresh == 0, axis=1).min()

    column, row, photo_width, photo_height = cv2.boundingRect(thresh)

    # axarr[0].imshow(thresh)
    # axarr[1].imshow(thresh[row:row+photo_height, column:column+photo_width])
    # axarr[2].imshow(img_in[row:row+photo_height, column:column+photo_width])
    # print((column, row, photo_width, photo_height))
    return (column, row, photo_width, photo_height)

# wycinanie zdjęcia z biełego tła oraz reskalowanie do 100x100
def image_crop(src_image, coordinates, res=(100, 100)):

    column, row, photo_width, photo_height = coordinates
    result_image = src_image[row:row + photo_height, column: column + photo_width]
    return cv2.resize(result_image, res)
    # return img_in[]
    # for rows in thresh:
    
    # img2, contours, hierarchy = cv2.findContours(thresh, 1, 2)
    # plt.imshow(img2)
    
    # print(np.where(thresh == 255) )
 
    # mask = 255 - mask

    # mask = cv2.GaussianBlur(mask, (0,0), sigmaX=2, sigmaY=2, borderType = cv2.BORDER_DEFAULT)

    # cnts = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    # cnt = sorted(cnts, key=cv2.contourArea)[-1]

    # x,y,w,h = cv2.boundingRect(cnt)
    # print(x, y, w, h)
    # dst = img_in[y:y+h, x:x+w]

    # result = img_in.copy()
    # result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
    # result[:, :, 3] = mask
    # img_in[:,:,3] = mask
    # plt.imshow(img_in)

    # dets = detector(img_in, upsample_num_times=1)

    # #print(dets)

    # img_result = img_in.copy()

    # for i, d in enumerate(dets):
    #     print("Detection {}: Left: {} Top: {} Right: {} Bottom: {} Confidence: {}".format(i, d.rect.left(), d.rect.top(), d.rect.right(), d.rect.bottom(), d.confidence))

    #     ((x1, y1), (x2, y2)) = (d.rect.left(), d.rect.top()), (d.rect.right(), d.rect.bottom())
    #     if y1 < 0:
    #         y1 = 0
    #     if x1 < 0:
    #         x1 = 0
    #     if x2 < 0:
    #         x2 = 0
    #     if y2 < 0:
    #         y2 = 0 


    # return ((x1, y1), (x2, y2))

# znajdywanie featerów na pieskach, tutaj każde zdjęcie zostaje podane operacji wyrównania histogramu, poczym cechy zostają wyciagnięte
def features_detector(src_image):

    ycrvb_img = cv2.cvtColor(src_image, cv2.COLOR_BGR2YCrCb)
    ycrvb_img[:, :, 0] = cv2.equalizeHist(ycrvb_img[:, :, 0])

    yuv_img = cv2.cvtColor(src_image, cv2.COLOR_BGR2YUV)
    yuv_img[:, :, 0] = cv2.equalizeHist(yuv_img[:, :, 0])

    equlized_img1 = cv2.cvtColor(ycrvb_img, cv2.COLOR_YCrCb2BGR)
    equlized_img2 = cv2.cvtColor(yuv_img, cv2.COLOR_YUV2BGR)

    # f, axarr = plt.subplots(1,4)
    # axarr[0].imshow(image)
    # axarr[1].imshow(ycrvb_img)
    # axarr[2].imshow(equlized_img1)
    # axarr[3].imshow(equlized_img2)

    dets = detector(equlized_img1, upsample_num_times=1)
    # if len(dets):
    if len(dets):
        shape = predictor(equlized_img1, dets.pop().rect)
        (forehead, right_ear, right_eye, nose, left_ear, left_eye) = face_utils.shape_to_np(shape)

        return (forehead, right_ear, right_eye, nose, left_ear, left_eye)
    else:
        return []

In [ ]:
# obraca zdjęcie tak, że linia oczu jest horyzontalnie
def img_rotate(src_image, face_features, res=(100, 100)):

    (forehead, right_ear, (x2, y2), nose, left_ear, (x1, y1)) = face_features
    
    slope = (y2 - y1) / (x2 - x1)
    rad = np.arctan2(y2 - y1, x2 - x1)
    angle = np.rad2deg(rad)

    cX = int(nose[0])
    cY = int(nose[1])
    rotationMatrix = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    rotatedImage = cv2.warpAffine(src_image, rotationMatrix, res)

    # plt.imshow(rotatedImage)

    return rotatedImage
    
def translate_image(src_image, translate_destination, feautres, feature_id=3, res=(100, 100)):
    tX, tY = translate_destination
    bX, bY = feautres[feature_id]

    M = np.float32([[1, 0, tX - bX],[0, 1,  tY - bY]])

    return_image = cv2.warpAffine(src_image, M, res)
    return return_image

def check_triangle_angles(face_features, max_angle = 70.0):
    (forehead, right_ear, (x1, y1), (x2, y2), left_ear, (x3, y3)) = face_features

    # calculate distance between dog eyes 
    a = np.sqrt((x1 - x3) ** 2 + (y1 - y3) ** 2)
    # calculate distance between dog left eye and nose 
    b = np.sqrt((x2 - x3) ** 2 + (y2 - y3) ** 2)
    # calculate distance between dog right eye and nose 
    c = np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
    
    cos_a = (b ** 2 + c ** 2 - a ** 2) / (2 * b * c)
    cos_b = (c ** 2 + a ** 2 - b ** 2) / (2 * a * c)
    cos_c = (a ** 2 + b ** 2 - c ** 2) / (2 * a * b)

    angle_a = np.rad2deg(np.arccos(cos_a))
    angle_b = np.rad2deg(np.arccos(cos_b))
    angle_c = np.rad2deg(np.arccos(cos_c))

    # print(angle_a, angle_b, angle_c)
    if angle_a < max_angle and angle_b < max_angle and angle_c < max_angle:
        return True

    return False
    

In [ ]:
def Face_detector(img):
    img_in = img

    img_in = cv2.cvtColor(img_in, cv2.COLOR_BGR2RGB)
    # img = cv2.resize(img, dsize=None, fx=0.5, fy=0.5)

    
    gray = cv2.cvtColor(img_in, cv2.COLOR_BGR2GRAY)
    th, threshed = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV) 

    cnts = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    cnt = sorted(cnts, key=cv2.contourArea)[-1]

    x,y,w,h = cv2.boundingRect(cnt)
    dst = img_in[y:y+h, x:x+w]

    plt.imshow(dst)

    dets = detector(img_in, upsample_num_times=1)

    #print(dets)

    img_result = img_in.copy()

    for i, d in enumerate(dets):
       # print("Detection {}: Left: {} Top: {} Right: {} Bottom: {} Confidence: {}".format(i, d.rect.left(), d.rect.top(), d.rect.right(), d.rect.bottom(), d.confidence))

        x1, y1 = d.rect.left(), d.rect.top()
 
        x2, y2 = d.rect.right(), d.rect.bottom()
        if y1 < 0:
            y1 = 0
        if x1 < 0:
            x1 = 0
        if x2 < 0:
            x2 = 0
        if y2 < 0:
            y2 = 0 

        #cv2.rectangle(img_result, pt1=(x1, y1), pt2=(x2, y2), thickness=2, color=(255,0,0), lineType=cv2.LINE_AA)
        
    shapes = []

    

    for i, d in enumerate(dets):
        shape = predictor(img_in, d.rect)
        shape = face_utils.shape_to_np(shape)
        # for i, p in enumerate(shape):
        shapes.append(shape)
            #cv2.putText(img_result, str(i), tuple(p), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
    # cv2.circle(img_result, center=shapes[4], radius=0.5, color=(0,0,255), thickness=-1, lineType=cv2.LINE_AA)
    # img_result = img_result[y1:y2,x1:x2]
    # print(img_result.shape)
    # img_result = cv2.resize(img_result,(100,100))
    #img_out = cv2.cvtColor(img_result, cv2.COLOR_RGB2BGR)
    #cv2.imwrite('img/%s_out%s' % (filename, ext), img_out)
    #plt.figure(figsize=(16, 16))
    # plt.imshow(img_result)
    try:
        return img_result, x1, x2, y1, y2
    except:
        return img_result

In [ ]:
## crop the dog before inputting into network. If no detection throw error

def Image_preprocessor(input_path,res=100):
    img = cv2.imread(input_path)
    x = np.expand_dims(x, axis=0)
    x = x.astype(np.float32)
    x/=255
    return x

            

In [ ]:
## load categories
CATEGORIES = ["Adult", "Senior", "Young"]

In [ ]:
def predDecoder(prediction):
    x = np.argmax(prediction)
    #print("Prediction: " + CATEGORIES[x])
    prob = prediction[x]
    print("Confidence: "+ str(prob))
    return (CATEGORIES[x])


In [ ]:
import shutil
## function to crop images with face detector 
## before inputting to network
## specify source folder and thrash folder
## incorrect photos will be copied to thrash 
## and removed from the source
## correct will be cropped and saved in the same
## folder
## run only ONCE on given folder
def all_img_loop(src,thrash):
  for filename in os.listdir(src):
    string_e=src+filename
    img = cv2.imread(string_e)
    IMG_SIZE =100
    try:
      x, x1, x2, y1, y2 = Face_detector(img)
      x = img[y1:y2,x1:x2]
      x = cv2.resize(x,(IMG_SIZE,IMG_SIZE))
      resized_image = x
      # save same img
      cv2.imwrite(string_e, resized_image)
    except: ## if file not detected, copy to trash folder and remove from the source
      wrong_address = thrash
      shutil.copy(string_e,wrong_address) ## move incorrect files to other folder
      os.remove(string_e)
      continue

In [ ]:
#all_img_loop('/home/data/data/kodeiri/ML_project/dogs datasets/Petfinder_All/Senior_experimental/')


In [ ]:
def metric_decoder(path,ground_truth, intensity=0.5, res=100):
  #iteration_number=len(os.listdir(path))
  iteration_number=5
  i=0
  cntr = 0
  error_counter = 0
  piclist=os.listdir(path)
  hm_sum = np.zeros((100, 100, 3, 1))
  for i in range (iteration_number):
    img_path = path+piclist[i]
    img = Image_preprocessor(img_path)
    if img.size == 0:
      error_counter = error_counter + 1
      continue ## skip the iteration if object not detected
      
    preds = model.predict(img)[0]
    predicted_class = predDecoder(preds)
    print("Ground_truth: "+ ground_truth)
    print("Prediction: " + predicted_class)
    if predicted_class == ground_truth:
      cntr=cntr+1
    conv_layer = model.get_layer(index=0)
    heatmap_model = tf.keras.models.Model([model.inputs], [conv_layer.output, model.output])
    
    with tf.GradientTape() as gtape:
        conv_output, predictions = heatmap_model(img)
        argmax=tf.argmax(predictions[0])
        loss = predictions[:, argmax]
        grads = gtape.gradient(loss, conv_output)
        pooled_grads = K.mean(grads, axis=(0,1,2))


    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    max_heat = np.max(heatmap)
    if max_heat == 0:
        max_heat = 1e-10
    heatmap /= max_heat
    
    img = cv2.imread(img_path) ## reload the original image without preprocessing to get facial features
    
    try:
            x, x1, x2, y1, y2 = Face_detector(img)
            x = img[y1:y2,x1:x2]
            x =  cv2.resize(x,(res,res))
            img = x
    except:
            print ('Error ocurred!')
            continue

    heatmap = heatmap.squeeze()
    #plt.matshow(heatmap)
    #plt.show()
    heatmap = cv2.resize(heatmap, (res, res))
    heatmap = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_HOT)
    ref_img = heatmap * intensity + img
    
    ref_img = cv2.resize(ref_img, (res, res))
    ref_img = cv2.cvtColor(ref_img.astype('float32'), cv2.COLOR_BGR2RGB)
    ref_img = ref_img/np.amax(ref_img)


    features = features_detector(img)
    if len(features):
      #print("Success1!")
      img = img_rotate(img, features, res=(res, res))
      heatmap = img_rotate(heatmap, features, res=(res, res))

    # wykrywanie featerów oraz translacja zdjęcia, tak aby nos znadował się w pozycji 50x50
    if len(features):
      #print("Success2!")
      img = translate_image(img, (50, 50), features, res=(res, res))
      heatmap = translate_image(heatmap, (50, 50), features, res=(res, res))
      hm_sum = np.append(hm_sum, heatmap.reshape((100, 100, 3, 1)), axis=3) ## to powinno iść w warunek niżej!

    # sprawdzenie czy znaki charakterystyczne pyska psa spełniają odpowiednie założenie - kąt maskymalny nie mozę przekraczać konkretnej wartosći
    if len(features) and check_triangle_angles(features, max_angle = 70.0):
      print("Success3!") ## czemu nie wchodzę w tę pętlę???
      #hm_sum = np.append(hm_sum, heatmap.reshape((100, 100, 3, 1)), axis=3)

    template_map = {

      "Adult" : Adult_template,
      "Young" : Young_template,
      "Senior" : Senior_template

    }
    heat_mean_val = heatmap.reshape((100, 100, 3, 1))

    b, g, r = cv2.split(heatmap)

    b_mean = b.mean().flatten()
    g_mean = g.mean().flatten()
    r_mean = r.mean().flatten()

    b_counts, b_bins = np.histogram(b_mean, range(257))
    g_counts, g_bins = np.histogram(g_mean, range(257))
    r_counts, r_bins = np.histogram(r_mean, range(257))


    b_equalize = cv2.equalizeHist(b)
    g_equalize = cv2.equalizeHist(g)
    r_equalize = cv2.equalizeHist(r)
    
    b_equalize_mean = b_equalize.mean().flatten()
    g_equalize_mean = g_equalize.mean().flatten()
    r_equalize_mean = r_equalize.mean().flatten()

    b_equalize_counts, b_equalize_bins = np.histogram(b_equalize_mean, range(257))
    g_equalize_counts, g_equalize_bins = np.histogram(g_equalize_mean, range(257))
    r_equalize_counts, r_equalize_bins = np.histogram(r_equalize_mean, range(257))


    heatmap_equalize = cv2.merge((b_equalize, g_equalize, r_equalize))      
    # hist = cv2.calcHist([heatmap],[0],None,[256],[0,256])
    # plt.hist(hist.ravel(),256,[0,256]); 
    ref_img = heatmap_equalize + img

    hist_fig, ax = plt.subplots(2, 2, figsize=(10, 10))
    ax[0][0].matshow(heatmap)
    ax[0][0].title.set_text('dog heatmap')
    # ax[0][1].hist(cv2.calcHist(heatmap, [0], None,[256], [0,256]), color='b')
    # ax[0][1].hist(cv2.calcHist(heatmap, [1], None,[256], [0,256]), color='g')
    # ax[0][1].hist(cv2.calcHist(heatmap, [2], None,[256], [0,256]), color='r')
    ax[0][1].bar(b_bins[:-1] - 0.5, b_counts, width=1, edgecolor='none', color='b')
    ax[0][1].bar(g_bins[:-1] - 0.5, g_counts, width=1, edgecolor='none', color='g')
    ax[0][1].bar(r_bins[:-1] - 0.5, r_counts, width=1, edgecolor='none', color='r')
    ax[0][1].title.set_text('heatmap histogram')
    ax[1][0].matshow(heatmap_equalize)
    # ax[1][0].title.set_text("equalized heatmap")
    # ax[1][1].hist(cv2.calcHist(heatmap_equalize, [0], None, [12000], [0,256]), color='b')
    # ax[1][1].hist(cv2.calcHist(heatmap_equalize, [1], None, [12000], [0,256]), color='g')
    # ax[1][1].hist(cv2.calcHist(heatmap_equalize, [2], None, [12000], [0,256]), color='r')
    ax[1][1].bar(b_equalize_bins[:-1] - 0.5, b_equalize_counts, width=1, edgecolor='none', color='b')
    ax[1][1].bar(g_equalize_bins[:-1] - 0.5, g_equalize_counts, width=1, edgecolor='none', color='g')
    ax[1][1].bar(r_equalize_bins[:-1] - 0.5, r_equalize_counts, width=1, edgecolor='none', color='r')
    ax[1][1].title.set_text("equalized heatmap histogram ")
    plt.show()

    # heat_mean_val = heat_mean_val.mean(axis=3)
    heat_mean_val = heat_mean_val.mean(axis=3)
    if predicted_class != ground_truth:
      print("Difference in prediction, calculating metric...")

      mean_difference_gt = template_map[ground_truth] - heat_mean_val
      mean_difference_prediction = template_map[predicted_class] - heat_mean_val
      
      gt_metric =  MSE(template_map[ground_truth].flatten(),heat_mean_val.flatten()) ## flattening required to calculate MSE
      pred_metric =  MSE(template_map[predicted_class].flatten(),heat_mean_val.flatten()) ## flattening required to calculate MSE
      metric_matrix = [gt_metric,pred_metric]
      print("Metric matrix: ")
      print(metric_matrix)
      
      fig, ax = plt.subplots(2, 2, figsize=(10, 10))  
      ax[0][0].matshow(mean_difference_gt)
      ax[0][0].title.set_text('Ground truth template mean - prediction heatmap')
      ax[0][1].matshow(mean_difference_prediction)
      ax[0][1].title.set_text('Prediction template mean -  prediction heatmap')
      ax[1][0].matshow(heatmap_equalize)
      ax[1][0].title.set_text("Prediction heatmap")
      ax[1][1].matshow(ref_img)
      ax[1][1].title.set_text("Prediction heatmap with image")
      plt.show()

  print ("Correct cases: " + str(cntr))
  print ("Total cases: " + str(iteration_number-error_counter))
  print ("Prediction accuracy: " + str(cntr/(iteration_number-error_counter)))
     # hm_sum = hm_sum[hm_sum != np.zeros((100, 100, 3))]
  # print(hm_sum[hm_sum != np.zeros((100, 100, 3))])
  # fig, ax = plt.subplots(1, 3, figsize=(20, 30))
  # ax[0].matshow(hm_sum.sum(axis=3))
  # ax[0].title.set_text('heatmaps sum')
  # ax[1].matshow(hm_sum.mean(axis=3))
  # ax[1].title.set_text('heatmaps mean')
  # # print(hm_sum.min().shape)
  # # hm_median = np.median((hm_sum - hm_sum.min()) / (hm_sum.max() - hm_sum.min()) , axis=3)
  # hm_median = np.median((hm_sum - hm_sum.mean()) / hm_sum.std() , axis=3)
  # ax[2].matshow(hm_median)
  # ax[2].title.set_text('heatmaps median')
  # # plt.matshow(hm_sum)
  # plt.show()
  return (mean_difference_gt, mean_difference_prediction)    

In [ ]:
senior_example = metric_decoder('/home/data/data/kodeiri/ML_project/test_photos_downloaded/Senior/',ground_truth="Senior")

In [ ]:
adult_example = metric_decoder('/home/data/data/kodeiri/ML_project/Mix_no_yolo_aug/Adult/',ground_truth="Adult")

## Here is part for pooling resulting imagaes

In [ ]:
class MinPooling2D(tf.keras.layers.MaxPool2D):

    def __init__(self, pool_size=(2, 2), strides=None, padding='valid', data_format=None, **kwargs):
        print(pool_size, strides, padding, data_format)
        super(tf.keras.layers.MaxPooling2D, self).__init__(pool_size=pool_size, strides=strides, padding=padding, data_format=data_format, pool_function=self.pooling_function, **kwargs)

    def pooling_function(inputs, ksize, strides, padding, data_format):
        return -keras.backend.pool2d(-inputs, ksize, strides, padding, data_format, pool_mode='max')

def min_pool(max_pool, input):
    return -max_pool(-input)

In [ ]:
# image_pooling_model = keras.Sequential()
# image_pooling_model =

image_size = (100, 100);

# avg_pool_2d = tf.keras.layers.AvgPool2D(pool_size=(10, 10), strides=(10, 10), padding='valid')
# min_pool_2d = MinPooling2D(pool_size=(10, 10), strides=(10, 10), padding='valid')
min_pool_2d_1 = tf.keras.layers.MaxPool2D(pool_size=(10, 10), strides=(2, 2), padding='valid')
# min_pool_2d_2 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='valid')

# output_pred = avg_pool_2d(senior_example[1])

output_gt = min_pool(min_pool_2d_1, senior_example[0].reshape((1, 100, 100, 3)))
plt.imshow(output_gt.numpy().reshape((output_gt.shape[1], output_gt.shape[2], output_gt.shape[3])))

# output_gt = min_pool(min_pool_2d_1, output_gt)
# plt.imshow(output_gt.numpy().reshape((output_gt.shape[1], output_gt.shape[2], output_gt.shape[3])))

# output_gt = min_pool(min_pool_2d_1, output_gt)
# plt.imshow(output_gt.numpy().reshape((output_gt.shape[1], output_gt.shape[2], output_gt.shape[3])))

# output_gt = min_pool(min_pool_2d_1, output_gt)
# plt.imshow(output_gt.numpy().reshape((output_gt.shape[1], output_gt.shape[2], output_gt.shape[3])))

## Footprints druga ścieżka

1. Zrobienie template heatmap dla danej choroby
2. Zrobienie template zdrowego psa
3. Pociąć zdjęcie wejścia na obszary ZROBIĆ ALE PÓŹNIEJ
4. Zrobienie funkcji obliczającej metryki (użyć metryk o skończonej dziedzinie) dla zdjęcia które chcemy zbadać
5. Obliczyć metryki dla zdjęcia wejściowego z template zdrowym i chorym
6. Analizować wyniki

In [18]:
## For development purposes, metrics will be calculated on adult template and young template
## When the algorithm will work, they will be replaced by valid heatmaps of predicted examples
def img_to_tensor(input_img):
    output_img = tf.convert_to_tensor(input_img)
    shapes = input_img.shape
    output_img = tf.reshape(output_img,[1,shapes[0],shapes[1],shapes[2]])
    return output_img


input_heatmap = img_to_tensor(Adult_template)
healthy_template = img_to_tensor(Young_template)
sick_template = img_to_tensor(Senior_template)



In [ ]:
def max_val_ssim (img1, img2=None, single_arg = False):
    # function returning max pixel value occuring
    # in a pair of images passed as 4D tf.tensors
    if single_arg==True:
        max1 = tf.math.reduce_max(img1,axis=(0, 1))
        max1 = tf.math.reduce_max(max1,axis=(0, 1))
        max_val = max1.numpy()
        return max_val    
    max1 = tf.math.reduce_max(img1,axis=(0, 1))
    max1 = tf.math.reduce_max(max1,axis=(0, 1))
    max2 = tf.math.reduce_max(img2,axis=(0, 1))
    max2 = tf.math.reduce_max(max2,axis=(0, 1))
    max_val = tf.math.maximum(max1,max2).numpy()
    return max_val

In [ ]:
from scipy.ndimage import correlate as corr


def metrics_calculator(input_pred,input_template):
    MSE_metric = MSE(input_template.numpy().flatten(),input_pred.numpy().flatten())
    SSIM_metric = tf.image.ssim(input_pred,input_template, max_val=max_val_ssim(input_pred,input_template)).numpy()[0]
    Correlation_metric = corr(input_pred,input_template)
    return  SSIM_metric, MSE_metric, Correlation_metric



In [1]:
sim, mse, cor = metrics_calculator(input_heatmap,healthy_template)
print(sim)
print(mse)
print(cor.squeeze().shape)



NameError: name 'metrics_calculator' is not defined

In [ ]:
ncor = cor/(max_val_ssim(cor,single_arg=True))

In [ ]:
plt.imshow(ncor.squeeze())
plt.show()